In [30]:
import pandas as pd
fdc_path = "../data/processed/fdc_data.h5"
rewe_path = "../data/processed/cleaned_rewe_dataset.csv"

fdc_df = pd.read_hdf(fdc_path)
rewe_df = pd.read_csv(rewe_path)
n_rows = 10
fdc_df = fdc_df.loc[:n_rows]
rewe_df = rewe_df.loc[:n_rows]

#fdc_df.columns = fdc_df.columns.to_flat_index()

rewe_df

pd.merge(fdc_df, rewe_df, left_index=True, right_index=True, how="inner")

Macronutrient                                   Micronutrient  \
   Carbohydrate [G] Energy [KCAL] Protein [G] fat [G]  Calcium [MG]   
0             28.80        443.40        7.35   33.20          41.0   
1              9.81        129.80        6.71    7.08         252.0   
2              5.51         31.03        0.83    0.63          11.0   
3              0.00          0.00        0.00    0.00         100.0   
4              4.11         24.11        1.04    0.39          36.0   
5             20.17        108.62        5.14    0.82          92.0   
6             10.37        100.92        6.76    3.60         252.0   
7              9.81         69.38        6.86    0.30         264.0   
8              9.30        170.86        6.55   11.94         246.0   
9              2.89        544.36       11.70   54.00          15.0   
10            16.20       1147.20       20.40  111.20         273.0   

                                                                  ...  \
   Choline [MG] Copper [MG] Folate [UG] Iron [MG] Magnesium [MG]  ...   
0          93.2       0.348        36.0      2.41           71.1  ...   
1          73.0       0.002         4.0      0.00           24.0  ...   
2          19.6       0.058        10.0      0.33           11.9  ...   
3           0.0       0.000         0.0      0.00            0.0  ...   
4           0.0       0.041         0.0      0.78           12.7  ...   
5           0.0       0.118       130.0      1.38           42.0  ...   
6          69.5       0.002         4.0      0.00           24.0  ...   
7          73.0       0.004         4.0      0.00           25.0  ...   
8          71.1       0.002         0.0      0.00           23.8  ...   
9           0.0       0.046         0.0      1.14           11.5  ...   
10        121.6       0.870        35.0      3.17          258.0  ...   

                                                                  \
   Vitamin C [MG] Vitamin D2 [UG] Vitamin D3 [UG] Vitamin E [MG]   
0             0.0             0.0            0.00           1.74   
1             0.0             0.0            2.23           0.06   
2            27.2             0.0            0.00           0.98   
3             0.0             0.0            0.00           0.00   
4             0.0             0.0            0.00           0.00   
5           144.6             0.0            0.00           0.30   
6             0.0             0.0            2.16           0.04   
7             0.0             0.0            2.20           0.00   
8             0.0             0.0            1.96           0.10   
9             0.0             0.0            0.00           0.51   
10            0.0             0.0            0.00          19.00   

                             \
   Vitamin K [UG] Zinc [MG]   
0            17.2      1.38   
1             0.0      0.86   
2             4.2      0.20   
3             0.0      0.00   
4             0.0      0.19   
5           204.0      0.84   
6             0.0      0.86   
7             0.0      0.90   
8             0.0      0.83   
9             0.0      2.06   
10            0.0      2.80   

                                    Non Nutrient Data  \
                                             FDC Name   
0                                  Hummus, commercial   
1   Milk, reduced fat, fluid, 2% milkfat, with add...   
2                                Tomatoes, grape, raw   
3                                Salt, table, iodized   
4   Beans, snap, green, canned, regular pack, drai...   
5                                       Broccoli, raw   
6   Milk, lowfat, fluid, 1% milkfat, with added vi...   
7   Milk, nonfat, fluid, with added vitamin A and ...   
8    Milk, whole, 3.25% milkfat, with added vitamin D   
9                         Frankfurter, beef, unheated   
10        Nuts, almonds, dry roasted, with salt added   

                                       \
                        Food Category   
0         Legumes a

In [2]:
# goals = calculate_nutrition(weight=88, height=180, age=24, calorie_adjustment=-200, activity_scale=0.5)
# data_path = "../data/processed/merged_dataset.csv"
# raw_df = pd.read_csv(data_path)
# df = process_data(raw_df)
# result_df = optimize_diet(df, goals)
# result_dfb
nutrition_columns = ["net_carbs", "fat", "protein", "fiber", "energy (kcal)"]
columns = ["Name", "Category", "Price (EUR/ 100g)"] + nutrition_columns
df["fiber"] = df["fiber"].fillna(0)
df = df.dropna()
df = df[~((df["net_carbs"] == 0) & (df["protein"] == 0) & (df["fat"] == 0))]
df[nutrition_columns] = df[nutrition_columns].astype(int)

# df = pd.read_csv("../data/raw/kaggle_satiety_dataset/satietyvsnutrivore.csv")
# df1 = pd.read_csv("../data/raw/kaggle_satiety_dataset/satietyindex.csv")

NameError: name 'df' is not defined